<a href="https://colab.research.google.com/github/priyadharshini13/pyspark/blob/main/PySpark_LearningSpark_Ch3_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 62.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=df6c986cfe5d83aa4e7bed645e30d91ca3b6f85c43f1d2175525b05b7acdbc23
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Ch3 - Common DF Operation').master('local[3]').getOrCreate()


# Common Dataframe operation

1. Using DataFrame reader
2. Using DataFrame Writer


In [3]:
import pandas as pd
# df_json = json.dumps(result)
emp_data = {"EMPID":{"0":1,"1":2,"2":3,"3":4,"4":5},"ENAME":{"0":"Deepika","1":"Vignesh","2":"Ganesh","3":"Haritha","4":"ilango"},"AGE":{"0":30,"1":27,"2":28,"3":29,"4":40},"SEX":{"0":"FEMALE","1":"MALE","2":"MALE","3":"FEMALE","4":"MALE"}}
extract_data_df = pd.DataFrame.from_dict(emp_data)
extract_data_df
# extract_data_df1 = extract_data_df["ENAME"].str.upper()

,EMPID,ENAME,AGE,SEX
0,1,Deepika,30,FEMALE
1,2,Vignesh,27,MALE
2,3,Ganesh,28,MALE
3,4,Haritha,29,FEMALE
4,5,ilango,40,MALE


In [4]:
from pyspark import SparkFiles

# url = 'https://raw.githubusercontent.com/jokecamp/FootballData/master/openFootballData/cities.csv'
url = 'https://raw.githubusercontent.com/databricks/LearningSparkV2/master/chapter3/data/sf-fire-calls.csv'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sf-fire-calls.csv"), header=True)
df.show()

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

In [5]:
# Defining schema
from pyspark.sql.types import *

# Programatic way to define schema
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
      StructField('UnitID', StringType(), True),
      StructField('IncidentNumber', IntegerType(), True),
      StructField('CallType', StringType(), True),
      StructField('CallDate', StringType(), True),
      StructField('WatchDate', StringType(), True),
      StructField('CallFinalDisposition', StringType(), True),
      StructField('AvailableDtTm', StringType(), True),
      StructField('Address', StringType(), True),
      StructField('City', StringType(), True),
      StructField('Zipcode', IntegerType(), True),
      StructField('Battalion', StringType(), True),
      StructField('StationArea', StringType(), True),
      StructField('Box', StringType(), True),
      StructField('OriginalPriority', StringType(), True),
      StructField('Priority', StringType(), True),
      StructField('FinalPriority', IntegerType(), True),
      StructField('ALSUnit', BooleanType(), True),
      StructField('CallTypeGroup', StringType(), True),
      StructField('NumAlarms', IntegerType(), True),
      StructField('UnitType', StringType(), True),
      StructField('UnitSequenceInCallDispatch', IntegerType(), True),
      StructField('FirePreventionDistrict', StringType(), True),
      StructField('SupervisorDistrict', StringType(), True),
      StructField('Neighborhood', StringType(), True),
      StructField('Location', StringType(), True),
      StructField('RowID', StringType(), True),
      StructField('Delay', FloatType(), True)])


In [6]:
url = 'https://raw.githubusercontent.com/databricks/LearningSparkV2/master/chapter3/data/sf-fire-calls.csv'
spark.sparkContext.addFile(url)
fire_df = spark.read.csv(SparkFiles.get('sf-fire-calls.csv'), header=True, schema=fire_schema)
fire_df.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [7]:
parquet_path = 'demo/fire'
fire_df.write.format("parquet").save(parquet_path)

parquet_table = 'fire1'
fire_df.write.format("parquet").saveAsTable(parquet_table) # stored inside spark-warehouse/

# Transformations and actions
 1. projections and filters
   Projection - a way to return only a specific row matching certain relational condition by using filters.

In [16]:
from pyspark.sql.functions import col
fire_df_filtered = fire_df.select("IncidentNumber", "AvailableDtTm", "CallType").where(col("CallType") != "Medical Incident")
fire_df_filtered.show()
# fire_df_filtered.explain()

+--------------+--------------------+--------------------+
|IncidentNumber|       AvailableDtTm|            CallType|
+--------------+--------------------+--------------------+
|       2003235|01/11/2002 01:51:...|      Structure Fire|
|       2003250|01/11/2002 04:16:...|        Vehicle Fire|
|       2003259|01/11/2002 06:01:...|              Alarms|
|       2003279|01/11/2002 08:03:...|      Structure Fire|
|       2003301|01/11/2002 09:46:...|              Alarms|
|       2003304|01/11/2002 09:58:...|              Alarms|
|       2003382|01/11/2002 02:59:...|      Structure Fire|
|       2003408|01/11/2002 04:09:...|      Structure Fire|
|       2003408|01/11/2002 04:09:...|      Structure Fire|
|       2003408|01/11/2002 04:09:...|      Structure Fire|
|       2003429|01/11/2002 05:17:...|Odor (Strange / U...|
|       2003453|01/11/2002 06:48:...|              Alarms|
|       2003497|01/11/2002 09:03:...|      Structure Fire|
|       2003554|01/12/2002 01:56:...|      Structure Fir

In [28]:
# Finding distinct callTypes 
  # countDistinct or count_distinct
from pyspark.sql.functions import *

# 1. Distinct on selected column
# fire_df_distinct1 = fire_df.select('CallType').where(col('CallType').isNotNull()).agg(countDistinct('CallType').alias('countDistinctCallType'))
# fire_df_distinct2 = fire_df.select('CallType').where(col('CallType').isNotNull()).agg(count_distinct('CallType'))
# # agg(countDistinct('CallType').alias('DistinctCallType'))
# fire_df_distinct1.show()
# fire_df_distinct2.show()

# 2. Distinct rows
fire_df_distinct_row = fire_df.distinct().count()
print('Distinct rows', fire_df_distinct_row)

Distinct rows 175296


In [25]:
#List distinct call type
fire_df_distinct_list = fire_df.select('CallType').where(col('CallType').isNotNull()).distinct().show(truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic